<a href="https://colab.research.google.com/github/NehaSontakk/BATH-Prokka-Comparison/blob/main/BATH_file_deduplication_checks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
e_value_threshold = 0.000001

In [ ]:
class filtering_operations:

  @staticmethod
  def e_value_filtering(df):
    return df.loc[df['E-value']<=e_value_threshold]

  @staticmethod
  def pos_neg_strand_filtering(df_filtered):
    #df should be post e_value_filteration
    df = filtering_operations.e_value_filtering(df_filtered)
    #Strand identification
    df['ali from'] = df['ali from'].astype(int)
    df['ali to'] = df['ali to'].astype(int)
    strand = []

    for index,row in df.iterrows():
      if row['ali from'] < row['ali to']:
        strand.append("+")
      elif row['ali from'] > row['ali to']:
        strand.append("-")

    df['strand'] = strand
    return df

  @staticmethod
  def get_specific_strand(df_raw,strand_info):
    df = filtering_operations.pos_neg_strand_filtering(df_raw)
    return df.loc[df['strand']==strand_info]


In [ ]:
class overlap_deduplications:

  ali_to_column = ""
  ali_from_column = ""

  #METHODS FOR 100% DEDUPLICATION

  @staticmethod
  def handle_group(group):
    #Now for each element in group
    min_e_value = group['E-value'].min()
    #find elements with lowest e-values
    group_min_e = group.loc[group['E-value'] == min_e_value]
    #if group contains a single row save it
    if len(group_min_e) == 1:
      return group_min_e
    #if group contains multiple rows with same e-values check min score
    elif len(group_min_e) > 1:
      return group_min_e.iloc[0:1]

  @staticmethod
  def deduplicate_full_overlaps(df):
    #100% Overlap Dedup
    print("Number of elements in relevant strand: ",df.shape)
    #Find all the duplicates
    duplicates = df.duplicated(subset=['target name',overlap_deduplications.ali_from_column, overlap_deduplications.ali_to_column], keep=False)
    #make a duplicates df
    duplicates_df = df[duplicates]
    #make a non duplicates df
    not_duplicate_df = df[~duplicates]
    print("Number of exact duplicates or homologs: ",duplicates_df.shape)
    #Sort by E-value and score
    duplicates_df1 = duplicates_df.sort_values(by=['E-value', 'score'], ascending=[True, False])
    deduplicated_remhomologs = duplicates_df1.groupby(['target name',overlap_deduplications.ali_from_column,overlap_deduplications.ali_to_column], group_keys=False).apply(overlap_deduplications.handle_group).reset_index(drop=True)
    #rejion after deduplication
    dedup_step1 = pd.concat([not_duplicate_df,deduplicated_remhomologs]).sort_values(['target name','ali from','E-value'])
    print("Number of elements after removing exact duplicates or homologs: ",dedup_step1.shape)
    print("Elements removed: ",df.shape[0] - dedup_step1.shape[0])
    return dedup_step1

  #METHODS FOR 70% DEDUPLICATION

  @staticmethod
  def calculate_overlap(hit_a, hit_b):
    #print("Overlap calculations done on columns:",overlap_deduplications.ali_from_column,overlap_deduplications.ali_to_column)
    start_a, end_a = hit_a[overlap_deduplications.ali_from_column], hit_a[overlap_deduplications.ali_to_column]
    start_b, end_b = hit_b[overlap_deduplications.ali_from_column], hit_b[overlap_deduplications.ali_to_column]

    overlap_length = max(0, min(end_a, end_b) - max(start_a, start_b) + 1)
    #print(min(end_a, end_b))
    #print(max(start_a, start_b))
    #print("Overlap length",overlap_length)

    length_a = end_a - start_a + 1
    length_b = end_b - start_b + 1

    #print("Length",length_a,length_b)
    if length_a > 0 and length_b > 0:
        overlap_perc_a = (overlap_length / length_a) * 100
        overlap_perc_b = (overlap_length / length_b) * 100
        return overlap_perc_a, overlap_perc_b
    else:
        return 0, 0

  @staticmethod
  def calculate_winner(hit_a, hit_b):
      overlap1, overlap2 = overlap_deduplications.calculate_overlap(hit_a, hit_b)
      print(f"Comparing:\nHit A: {hit_a}\nHit B: {hit_b}\nOverlap1: {overlap1}%, Overlap2: {overlap2}%")

      # Check for significant overlap
      if overlap1 >= 70 or overlap2 >= 70:
          # Determine which hit to retain based on E-value or sequence length on tie
          if hit_a['E-value'] < hit_b['E-value']:
              print("Choosing Hit A based on E-value")
              return hit_a
          elif hit_a['E-value'] > hit_b['E-value']:
              print("Choosing Hit B based on E-value")
              return hit_b
          else:  # E-values are tied, check the sequence length
              if hit_a['seq len'] > hit_b['seq len']:
                  print("E-values tied. Choosing Hit A based on larger sequence length")
                  return hit_a
              else:
                  print("E-values tied. Choosing Hit B based on larger sequence length")
                  return hit_b
      else:
          print("No significant overlap or both hits retained")
          return hit_a, hit_b

  @staticmethod
  def handle_70_overlaps(group):
    items = []
    for index, row in group.iterrows():
      row_tuple = row.to_dict()
      items.append(row_tuple)

    i = 0
    all_winners = []

    while i<len(items)-1:
      current = items[i]
      next_item = items[i+1]
      print("Begining Comparison: ",i)
      #print(current,"\n",next_item)
      result = overlap_deduplications.calculate_winner(current, next_item)

      #If result is two items append the current item to winners list since overlap of current item with next item is less than 70% so we want to keep both
      if isinstance(result,tuple):
        if result[0] not in all_winners:
          all_winners.append(result[0])
          #print("Tuple returned!")
          #print("Appending: ",result[0])
          #print("All winners currently: ",all_winners)
        #else:
        #  print("Result already in all winners, no appending!")
        i += 1
      else:
        if result == current:
          #print("Result is ",i," element and not ",i+1," popping: ",items[i+1])
          items.pop(i+1)
        else:
          i += 1

      if i >= len(items) - 1 and items[-1] not in all_winners:
        all_winners.append(items[-1])

    return pd.DataFrame(all_winners)

  #METHODS FOR LESS THAN 70% DEDUPLICATION
  @staticmethod
  def calculate_below_70_changes(hit_a, hit_b):
    overlap1, overlap2 = overlap_deduplications.calculate_overlap(hit_a, hit_b)
    #print("Overlap 1: ",overlap1)
    if (0.01 <= overlap1 < 70) and (0.01 <= overlap2 < 70):
      if hit_a['E-value'] < hit_b['E-value']:
        #print("Case A: A dominates")
        #print(hit_a)
        #print(hit_b)
        hit_b['ali from'] = hit_a['ali to'] + 1
        #print(hit_b)
        return hit_a, hit_b
      else:
        #print("Case B: B dominates")
        #print(hit_a)
        #print(hit_b)
        hit_a['ali to'] = hit_b['ali from'] - 1
        #print(hit_a)
        #print(hit_b)
        return hit_a, hit_b
    else:
      return hit_a,hit_b

  @staticmethod
  def below_70_overlap(group):
    items = []
    for index, row in group.iterrows():
      row_tuple = row.to_dict()
      items.append(row_tuple)

    for i in range(0,len(items)-1):
      element1, element2 = overlap_deduplications.calculate_below_70_changes(items[i],items[i+1])
      items[i] = element1
      items[i+1] = element2

    return pd.DataFrame(items)

  # FINAL METHOD TO CALL
  @staticmethod
  def choose_strand_operations(df):
    if df['strand'].values[0] == '-':
      print("Processing negative strand data...")
      #create the flip columns
      df['ali from flip'] = df['ali to']
      df['ali to flip'] = df['ali from']
      overlap_deduplications.ali_to_column = 'ali to flip'
      overlap_deduplications.ali_from_column = 'ali from flip'
      #100% dedup
      df = df.sort_values(by=['ali from flip','E-value'], ascending=[True, True])
      print(df)
      step1_dedup = overlap_deduplications.deduplicate_full_overlaps(df)
      print("Overlap calculations done on columns:",overlap_deduplications.ali_from_column,overlap_deduplications.ali_to_column)
      print("100% deduplication done.")
      #70% Overlap
      step2_dedup = step1_dedup.groupby(['target name'], group_keys=False).apply(overlap_deduplications.handle_70_overlaps).reset_index(drop=True)
      print("Overlap calculations done on columns:",overlap_deduplications.ali_from_column,overlap_deduplications.ali_to_column)
      print("70% deduplication done.")
      #Less than 70% overlap
      step3_dedup = step2_dedup.groupby(['target name'], group_keys=False).apply(overlap_deduplications.below_70_overlap).reset_index(drop=True)
      print("Less than 70% deduplication done.")
      #return something final
      return step3_dedup
    elif df['strand'].values[0] == '+':
      print("Processing positive strand data...")
      overlap_deduplications.ali_to_column = 'ali to'
      overlap_deduplications.ali_from_column = 'ali from'
      #100% dedup
      df = df.sort_values(by=['ali from','E-value'], ascending=[True, True])
      print(df)
      step1_dedup = overlap_deduplications.deduplicate_full_overlaps(df)
      print("Overlap calculations done on columns:",overlap_deduplications.ali_from_column,overlap_deduplications.ali_to_column)
      print("100% deduplication done.")
      #70% Overlap
      step2_dedup = step1_dedup.groupby(['target name'], group_keys=False).apply(overlap_deduplications.handle_70_overlaps).reset_index(drop=True)
      print("Overlap calculations done on columns:",overlap_deduplications.ali_from_column,overlap_deduplications.ali_to_column)
      print("70% deduplication done.")
      #Less than 70% overlap
      step3_dedup = step2_dedup.groupby(['target name'], group_keys=False).apply(overlap_deduplications.below_70_overlap).reset_index(drop=True)
      print("Less than 70% deduplication done.")
      #return something final
      return step3_dedup




In [ ]:
#Do it for all BATH files we have
from glob import glob

for i in glob("/content/drive/MyDrive/Lab Work/Parkinsons_Data/Bath_Output/*.tbl"):
  bathout = pd.read_table("/content/drive/MyDrive/Lab Work/Parkinsons_Data/Bath_Output/DNA_Bacteria_kingdom_sprot.tbl",sep="\s+",header=None,skiprows=2,skipfooter=8)
  col_names = ['target name', 'accession', 'query name', 'accession1', 'hmm len', 'hmm from', 'hmm to', 'seq len', 'ali from', 'ali to', 'env from', 'env to', 'E-value', 'score', 'bias', 'shifts', 'stops', 'pipe', 'description of target', 'extra']
  bathout.rename(columns=dict(zip(bathout.columns, col_names)), inplace=True)
  df_neg = filtering_operations.get_specific_strand(bathout,"-")
  neg_deduped = overlap_deduplications.choose_strand_operations(df_neg)
  df_pos = filtering_operations.get_specific_strand(bathout,"+")
  pos_deduped = overlap_deduplications.choose_strand_operations(df_pos)
  bath_file_deduplication = pd.concat([pos_deduped,neg_deduped])
  bath_file_deduplication['origin'] = str(i.split("/")[-1].split(".tbl")[0])
  bath_file_deduplication.to_excel(str(i.split("/")[-1].split(".tbl")[0]+".xlsx"),index=False)

<ipython-input-5-718c2c694ba7>:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  bathout = pd.read_table("/content/drive/MyDrive/Lab Work/Parkinsons_Data/Bath_Output/DNA_Bacteria_kingdom_sprot.tbl",sep="\s+",header=None,skiprows=2,skipfooter=8)
<ipython-input-3-830c4c50e74a>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ali from'] = df['ali from'].astype(int)
<ipython-input-3-830c4c50e74a>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/use

Streaming output truncated to the last 5000 lines.
Comparing:
Hit A: {'target name': 'k127_748425', 'accession': '-', 'query name': 'P9WG39', 'accession1': '-', 'hmm len': 547, 'hmm from': 15, 'hmm to': 532, 'seq len': 125518, 'ali from': 92966, 'ali to': 91374, 'env from': 92999, 'env to': 91347, 'E-value': 2.6000000000000002e-37, 'score': 140.7, 'bias': 0.2, 'shifts': 0, 'stops': 0, 'pipe': 'std', 'description of target': '-', 'strand': '-', 'ali from flip': 91374, 'ali to flip': 92966}
Hit B: {'target name': 'k127_748425', 'accession': '-', 'query name': 'Q9LCV9', 'accession1': '-', 'hmm len': 573, 'hmm from': 13, 'hmm to': 260, 'seq len': 125518, 'ali from': 92975, 'ali to': 92235, 'env from': 93002, 'env to': 92211, 'E-value': 2.8e-09, 'score': 46.1, 'bias': 0.1, 'shifts': 0, 'stops': 0, 'pipe': 'std', 'description of target': '-', 'strand': '-', 'ali from flip': 92235, 'ali to flip': 92975}
Overlap1: 45.951035781544256%, Overlap2: 98.78542510121457%
Choosing Hit A based on E-valu

<ipython-input-3-830c4c50e74a>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ali from'] = df['ali from'].astype(int)
<ipython-input-3-830c4c50e74a>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ali to'] = df['ali to'].astype(int)
<ipython-input-3-830c4c50e74a>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable

Streaming output truncated to the last 5000 lines.
Overlap1: 97.48953974895397%, Overlap2: 40.103270223752155%
Choosing Hit B based on E-value
Begining Comparison:  52
Comparing:
Hit A: {'target name': 'k127_533492', 'accession': '-', 'query name': 'Q5KWH6', 'accession1': '-', 'hmm len': 590, 'hmm from': 2, 'hmm to': 573, 'seq len': 131788, 'ali from': 95464, 'ali to': 97206, 'env from': 95461, 'env to': 97245, 'E-value': 9.5e-113, 'score': 390.5, 'bias': 0.7, 'shifts': 0, 'stops': 0, 'pipe': 'std', 'description of target': '-', 'strand': '+'}
Hit B: {'target name': 'k127_533492', 'accession': '-', 'query name': 'Q9WYA3', 'accession1': '-', 'hmm len': 557, 'hmm from': 2, 'hmm to': 245, 'seq len': 131788, 'ali from': 95464, 'ali to': 96231, 'env from': 95461, 'env to': 96345, 'E-value': 7.8e-34, 'score': 130.3, 'bias': 0.9, 'shifts': 0, 'stops': 0, 'pipe': 'std', 'description of target': '-', 'strand': '+'}
Overlap1: 44.061962134251296%, Overlap2: 100.0%
Choosing Hit A based on E-value


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

<ipython-input-5-718c2c694ba7>:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  bathout = pd.read_table("/content/drive/MyDrive/Lab Work/Parkinsons_Data/Bath_Output/DNA_Bacteria_kingdom_sprot.tbl",sep="\s+",header=None,skiprows=2,skipfooter=8)
<ipython-input-3-830c4c50e74a>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-ve

Streaming output truncated to the last 5000 lines.
Choosing Hit A based on E-value
Begining Comparison:  43
Comparing:
Hit A: {'target name': 'k127_39946', 'accession': '-', 'query name': 'O31711', 'accession1': '-', 'hmm len': 230, 'hmm from': 1, 'hmm to': 222, 'seq len': 74930, 'ali from': 74527, 'ali to': 73871, 'env from': 74527, 'env to': 73853, 'E-value': 3.1999999999999996e-49, 'score': 181.4, 'bias': 0.0, 'shifts': 0, 'stops': 0, 'pipe': 'std', 'description of target': '-', 'strand': '-', 'ali from flip': 73871, 'ali to flip': 74527}
Hit B: {'target name': 'k127_39946', 'accession': '-', 'query name': 'P75264', 'accession1': '-', 'hmm len': 586, 'hmm from': 12, 'hmm to': 120, 'seq len': 74930, 'ali from': 74527, 'ali to': 74180, 'env from': 74539, 'env to': 74171, 'E-value': 3.6e-09, 'score': 48.8, 'bias': 0.0, 'shifts': 0, 'stops': 0, 'pipe': 'std', 'description of target': '-', 'strand': '-', 'ali from flip': 74180, 'ali to flip': 74527}
Overlap1: 52.96803652968036%, Overlap2

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)




Begining Comparison:  67
Comparing:
Hit A: {'target name': 'k127_795063', 'accession': '-', 'query name': 'Q8DCQ8', 'accession1': '-', 'hmm len': 699, 'hmm from': 8, 'hmm to': 696, 'seq len': 57992, 'ali from': 21313, 'ali to': 19241, 'env from': 21337, 'env to': 19232, 'E-value': 2.7e-236, 'score': 796.8, 'bias': 0.1, 'shifts': 0, 'stops': 0, 'pipe': 'std', 'description of target': '-', 'strand': '-', 'ali from flip': 19241, 'ali to flip': 21313}
Hit B: {'target name': 'k127_795063', 'accession': '-', 'query name': 'Q99V72', 'accession1': '-', 'hmm len': 520, 'hmm from': 7, 'hmm to': 193, 'seq len': 57992, 'ali from': 21316, 'ali to': 20756, 'env from': 21331, 'env to': 20663, 'E-value': 4.2999999999999996e-24, 'score': 97.8, 'bias': 0.0, 'shifts': 0, 'stops': 0, 'pipe': 'std', 'description of target': '-', 'strand': '-', 'ali from flip': 20756, 'ali to flip': 21316}
Overlap1: 26.91751085383502%, Overlap2: 99.46524064171123%
Choosing Hit A based on E-value
Begining Comparison:  67
Co

<ipython-input-3-830c4c50e74a>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ali from'] = df['ali from'].astype(int)
<ipython-input-3-830c4c50e74a>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ali to'] = df['ali to'].astype(int)


In [ ]:
list_of_dfs = []
for i in glob("/content/*.xlsx"):
  list_of_dfs.append(pd.read_excel(i))

In [ ]:
bathout = pd.concat(list_of_dfs)
df_neg = filtering_operations.get_specific_strand(bathout,"-")
neg_deduped = overlap_deduplications.choose_strand_operations(df_neg)
df_pos = filtering_operations.get_specific_strand(bathout,"+")
pos_deduped = overlap_deduplications.choose_strand_operations(df_pos)
complete_bath_deduplication = pd.concat([pos_deduped,neg_deduped])

In [ ]:
complete_bath_deduplication.to_excel("bin82_BATH_deduplicated.xlsx")

In [ ]:
complete_bath_deduplication.shape

(890, 23)

In [ ]:
import pandas as pd

# Define the path to the newly uploaded file
file_path = '/content/bin82_BATH_deduplicated.xlsx'

df = pd.read_excel(file_path, header=None, skiprows=2)
df.head()
col_names = [
    'target name', 'accession', 'query name', 'accession1', 'hmm len', 'hmm from',
    'hmm to', 'seq len', 'ali from', 'ali to', 'env from', 'env to', 'E-value',
    'score', 'bias', 'shifts', 'stops', 'pipe', 'description of target'
]

# Rename the columns in the DataFrame
df.columns = col_names

# Perform strand specific filtering and deduplication
df_neg = filtering_operations.get_specific_strand(df, "-")
#neg_deduped = overlap_deduplications.choose_strand_operations(df_neg)

df_pos = filtering_operations.get_specific_strand(df, "+")
pos_deduped = overlap_deduplications.choose_strand_operations(df_pos)

# Combine negative and positive strand results
#bath_file_deduplication = pd.concat([pos_deduped, neg_deduped])
bath_file_deduplication['origin'] = file_path.split("/")[-1].split(".txt")[0]

# Save the deduplicated results to an Excel file
output_path = '/content/' + file_path.split("/")[-1].split(".txt")[0] + "_deduplicated.xlsx"
bath_file_deduplication.to_excel(output_path, index=False)

print(f"Data processed and saved to {output_path}")


ValueError: Length mismatch: Expected axis has 24 elements, new values have 19 elements